In [115]:
ls

bball-ref.js      cleanGameLogs.py  finalGameLog.py   gameLogs.ipynb    includes/         node_modules/     players/


In [116]:
import pandas as pd
import numpy as np

logs = 'players/2015/alllogs.csv'

games = pd.read_csv(logs)

In [117]:
games = games.rename(columns={'Unnamed: 5': 'Home'})
games = games.rename(columns={'Unnamed: 7': 'ScoreDiff'})
games = games.drop(['Rk', 'G'], axis=1)

In [118]:
games.Home = games.Home.fillna(value=1)
games.Home = games.Home.replace(to_replace='@', value=0)

In [119]:
games['ScoreDiff'] = games.apply(lambda row: (int(str(row['ScoreDiff']).split('(')[1].split(')')[0])), axis=1)

In [120]:
#games.dtypes

In [121]:
games['fptsFT'] = games.apply(lambda row: (row['FT']), axis=1)
games['fpts2P'] = games.apply(lambda row: (row['FG']*2), axis=1)
games['fpts3P'] = games.apply(lambda row: (row['3P']*3), axis=1)
games['fptsRebounds'] = games.apply(lambda row: (row['TRB']*1.2), axis=1)
games['fptsAssists'] = games.apply(lambda row: (row['AST']*1.5), axis=1)
games['fptsBlocks'] = games.apply(lambda row: (row['BLK']*2), axis=1)
games['fptsSteals'] = games.apply(lambda row: (row['STL']*2), axis=1)
games['fptsTurnovers'] = games.apply(lambda row: (row['TOV']*-1), axis=1)

games['fpts'] = games['fptsFT'] + games['fpts2P'] + games['fptsRebounds'] + games['fptsAssists'] + \
games['fptsBlocks'] + games['fptsSteals'] + games['fptsTurnovers']

In [122]:
games

,Date,Age,Tm,Home,Opp,ScoreDiff,GS,MP,FG,FGA,...,pname,fptsFT,fpts2P,fpts3P,fptsRebounds,fptsAssists,fptsBlocks,fptsSteals,fptsTurnovers,fpts
0,2014-10-29,24-023,NYK,1,CHI,-24,0,21:28,3,6,...,acyqu01,0,6,0,7.2,1.5,2,0,-2,14.7
1,2014-10-30,24-024,NYK,0,CLE,5,1,22:27,4,7,...,acyqu01,0,8,0,12.0,3.0,2,0,-2,23.0
2,2014-11-02,24-027,NYK,1,CHO,3,1,20:20,1,4,...,acyqu01,0,2,0,3.6,3.0,0,0,0,8.6
3,2014-11-05,24-030,NYK,0,DET,-3,0,13:42,0,1,...,acyqu01,0,0,0,7.2,1.5,2,0,-2,8.7
4,2014-11-07,24-032,NYK,0,BRK,-11,0,22:14,3,7,...,acyqu01,1,6,0,10.8,0.0,0,0,-3,14.8
5,2014-11-08,24-033,NYK,0,ATL,-7,0,13:34,1,1,...,acyqu01,0,2,0,3.6,0.0,0,0,-1,4.6
6,2014-11-12,24-037,NYK,1,ORL,-2,1,20:12,1,1,...,acyqu01,0,2,0,4.8,1.5,0,0,-2,6.3
7,2014-11-14,24-039,NYK,1,UTA,-2,0,19:39,2,2,...,acyqu01,2,4,0,2.4,0.0,0,0,0,8.4
8,2014-11-16,24-041,NYK,1,DEN,16,1,31:10,3,7,...,acyqu01,2,6,0,9.6,4.5,0,6,0,28.1
9,2014-11-19,24-044,NYK,0,MIN,-16,1,32:18,4,8,...,acyqu01,1,8,0,9.6,0.0,0,0,-3,15.6


In [123]:
games.describe()

,Home,ScoreDiff,GS,FG,FGA,FG%,3P,3PA,3P%,FT,...,+/-,fptsFT,fpts2P,fpts3P,fptsRebounds,fptsAssists,fptsBlocks,fptsSteals,fptsTurnovers,fpts
count,12998.000000,12998.000000,12998.000000,12998.000000,12998.000000,12410.000000,12998.000000,12998.000000,8990.000000,12998.000000,...,12994.000000,12998.000000,12998.000000,12998.000000,12998.000000,12998.000000,12998.000000,12998.000000,12998.000000,12998.000000
mean,0.504231,0.419911,0.450531,3.492230,7.965995,0.430735,0.872596,2.476381,0.327273,1.669334,...,0.035401,1.669334,6.984459,2.617787,4.620526,3.257578,0.756116,1.523311,-1.340822,17.470503
std,0.500001,14.134076,0.497566,2.765219,5.376600,0.230757,1.212164,2.537861,0.298644,2.234343,...,10.711715,2.234343,5.530438,3.636491,4.009160,3.577376,1.524968,2.028921,1.425759,11.782061
min,0.000000,-53.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-10.000000,-3.000000
25%,0.000000,-9.000000,0.000000,1.000000,4.000000,0.300000,0.000000,0.000000,0.000000,0.000000,...,-7.000000,0.000000,2.000000,0.000000,1.200000,0.000000,0.000000,0.000000,-2.000000,8.500000
50%,1.000000,1.000000,0.000000,3.000000,7.000000,0.429000,0.000000,2.000000,0.333000,1.000000,...,0.000000,1.000000,6.000000,0.000000,3.600000,1.500000,0.000000,0.000000,-1.000000,15.800000
75%,1.000000,10.000000,1.000000,5.000000,11.000000,0.556000,1.000000,4.000000,0.500000,3.000000,...,7.000000,3.000000,10.000000,3.000000,6.000000,4.500000,2.000000,2.000000,0.000000,24.700000
max,1.000000,53.000000,1.000000,17.000000,37.000000,1.000000,10.000000,16.000000,1.000000,18.000000,...,45.000000,18.000000,34.000000,30.000000,30.000000,25.500000,18.000000,16.000000,0.000000,77.200000
